# JSSP comparação entre euristicas Classicas: codigo fonte

## importação de libs

In [ ]:
# %pip install mealpy
# %pip install numpy
%pip install -r ../requirements.txt


In [1]:
import os
import importlib.util
from classes.jssp import jssp
import numpy as np
from classes.jssp import jssp
from mealpy import SA
from mealpy.utils.space import FloatVar
import matplotlib.pyplot as plt
import time

## funções adicionais

In [2]:
# importar casos de teste
def import_tests_cases(nome_dict : str) -> dict:

    caminho_absoluto = os.path.abspath("../tests/test1.py")

    spec = importlib.util.spec_from_file_location("modulo_temp", caminho_absoluto)
    
    if spec is None or spec.loader is None:
        raise ImportError(f"Não foi possível carregar o módulo do arquivo: {caminho_absoluto}")
    
    modulo = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(modulo)

    return getattr(modulo, nome_dict)

def decode_solution(solution_vec, operations):
    import numpy as np
    order = np.argsort(solution_vec)

    machine_available = {}
    job_available = {}

    print("Ordem com máquina usada:")

    for i, idx in enumerate(order):
        op = operations[idx]
        job = op["job"]

        # Se quiser a regra de escolha original, substitua esta linha:
        # machine = op["machines"][0]

        # Exemplo: escolher a máquina que fica disponível primeiro
        machine = min(op["machines"], key=lambda m: machine_available.get(m, 0)) # verificar com pamela

        start_time = max(machine_available.get(machine, 0), job_available.get(job, 0))
        end_time = start_time + op["duration"]

        machine_available[machine] = end_time
        job_available[job] = end_time

        print(f"{i+1}: {op['job']}, Máquina {machine}, Duração {op['duration']}, Início {start_time}, Fim {end_time}")

def make_fitness_function(instance: jssp):

    # Gera uma lista de dicionários com informações sobre cada operação
    # Isso é feito uma única vez fora da função fitness por eficiência
    operations = instance.get_flattened_operations()

    # Esta é a função que o mealpy usará para avaliar cada solução
    def fitness(solution):
        import numpy as np

        # Ordena os índices das operações com base nos valores do vetor de entrada
        # Isso define a ordem de execução das operações
        priority_order = np.argsort(solution)

        # Dicionários para acompanhar quando cada máquina e cada job estarão disponíveis
        machine_available = {}  # ex: {1: 5} → máquina 1 estará livre no tempo 5
        job_available = {}      # ex: {"job_1": 4} → job_1 pode iniciar próxima operação no tempo 4
        end_times = []          # armazenará o tempo de término de cada operação

        # Executa as operações na ordem definida pelo vetor de prioridade
        for idx in priority_order:
            op = operations[idx]  # Recupera a operação pelo índice

            job = op["job"]               # Nome do job (ex: "job_1")
            machine = min(op["machines"], key=lambda m: machine_available.get(m, 0)) # Seleciona a máquina disponível mais cedo
            duration = op["duration"]     # Duração da operação

            # O início da operação depende da disponibilidade da máquina e do job
            start_time = max(
                machine_available.get(machine, 0),  # se máquina não usada ainda, começa em 0
                job_available.get(job, 0)           # se job ainda não iniciou, começa em 0
            )

            end_time = start_time + duration  # Tempo em que a operação termina

            # Atualiza os dicionários com a nova disponibilidade após a operação
            machine_available[machine] = end_time
            job_available[job] = end_time

            end_times.append(end_time)  # Salva o tempo final da operação

        # O makespan é o tempo máximo de término entre todas as operações
        return max(end_times),  # Retorna como tupla, como o mealpy exige

    # Retorna a função de fitness já configurada com os dados da instância
    return fitness




## Configurações gerais

In [26]:
# dados para os testes iniciais
data = import_tests_cases("best")
instance = jssp(data)
fitness_func = make_fitness_function(instance)
num_ops = len(instance.get_flattened_operations())

# Definindo o problema
problem = {
    "obj_func": fitness_func,
    "bounds": [FloatVar(lb=0.0, ub=1.0) for _ in range(num_ops)],
    "minmax": "min",
    "log_to": None,
}


## Simulated annealing

In [29]:
times = []
solutions = []
model = SA.OriginalSA(epoch=100)
for _ in range(30):
    start_time = time.time()
    g_best = model.solve(problem)
    end_time = time.time()
    times.append([end_time - start_time, g_best.id])
    solutions.append(g_best)




In [28]:
print("Soluçao mais rapida:", solutions[times.index(min(times))])
best = min(solutions, key=lambda s: s.target.fitness)
print(f"ID: {best.id}, Fitness: {best.target.fitness}, Solution: {best.solution}")



Soluçao mais rapida: id: 86535, target: Objectives: [1], Fitness: 1.0, solution: [0.16090017 0.41270968]
ID: 76864, Fitness: 1.0, Solution: [0.49681214 0.66830162]
